In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
print({
    "uuid": "esd42ujTT0C1V0K5QKvA9Q",
    "viewconf": "{\n  \"editable\": true,\n  \"trackSourceServers\": [\n    \"/api/v1\",\n    \"http://higlass.io/api/v1\"\n  ],\n  \"exportViewUrl\": \"/api/v1/viewconfs\",\n  \"views\": [\n    {\n      \"initialXDomain\": [\n        1621867138.8013206,\n        1624295541.6087596\n      ],\n      \"initialYDomain\": [\n        1620849673.912563,\n        1623205466.669282\n      ],\n      \"tracks\": {\n        \"top\": [\n          {\n            \"type\": \"horizontal-stacked-bar\",\n            \"uid\": \"PtVf0DYWTwCnGrGW9AkqWg\",\n            \"tilesetUid\": \"WmHBFc78TmOgTCIePEwawQ\",\n            \"server\": \"https://resgen.io/api/v1\",\n            \"options\": {\n              \"labelPosition\": \"topLeft\",\n              \"labelColor\": \"black\",\n              \"labelTextOpacity\": 0.4,\n              \"valueScaling\": \"exponential\",\n              \"trackBorderWidth\": 0,\n              \"trackBorderColor\": \"black\",\n              \"backgroundColor\": \"white\",\n              \"barBorder\": true,\n              \"sortLargestOnTop\": true,\n              \"selectRows\": null,\n              \"name\": \"Epilogos (mm10)\",\n              \"colorScale\": [\n                \"#0e6f37\",\n                \"#c7e4c0\",\n                \"#cdcdcd\",\n                \"#41ac5e\",\n                \"#f3eb1a\",\n                \"#f3eb1a\",\n                \"#faf8c8\",\n                \"#808080\",\n                \"#808080\",\n                \"#0454a3\",\n                \"#deecf7\",\n                \"#4290cf\",\n                \"#f48c8f\",\n                \"#fde2e5\",\n                \"#ffffff\"\n              ]\n            },\n            \"width\": 468,\n            \"height\": 24\n          },\n          {\n            \"uid\": \"KgmNtAD8RM-FZPXiTGQuNw\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"chromosome-labels\",\n                \"uid\": \"Lm1q786QQgixJMOm5sLjWA\",\n                \"tilesetUid\": \"O8tQOpn0RjiFrUBmrhph4A\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"color\": \"#808080\",\n                  \"stroke\": \"#ffffff\",\n                  \"fontSize\": 12,\n                  \"fontIsLeftAligned\": false,\n                  \"showMousePosition\": false,\n                  \"mousePositionColor\": \"#000000\"\n                },\n                \"width\": 468,\n                \"height\": 30\n              }\n            ],\n            \"height\": 30,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"JePzgKgsSeS9tRGqCJDZ8Q\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"horizontal-gene-annotations\",\n                \"uid\": \"cPxh3DfvS8mOWHPghMz2HQ\",\n                \"tilesetUid\": \"DVuS61VhTG6DxKhlsTssTQ\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"fontSize\": 8,\n                  \"labelColor\": \"black\",\n                  \"labelBackgroundColor\": \"#ffffff\",\n                  \"labelPosition\": \"hidden\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"minHeight\": 24,\n                  \"plusStrandColor\": \"blue\",\n                  \"minusStrandColor\": \"red\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"showMousePosition\": false,\n                  \"mousePositionColor\": \"#000000\",\n                  \"geneAnnotationHeight\": 8,\n                  \"geneLabelPosition\": \"outside\",\n                  \"geneStrandSpacing\": 4,\n                  \"name\": \"Gene Annotations (mm10)\"\n                },\n                \"width\": 468,\n                \"height\": 81\n              }\n            ],\n            \"height\": 81,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"Ic97pmWoT1uJcTJyBr-JQw\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"ZI5iCXp1Ryq5jw0AlaMGog\",\n                \"tilesetUid\": \"YeXV8q2TQ8-jSgsUa3d1HA\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"gold\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"H3K27ac\"\n                },\n                \"width\": 468,\n                \"height\": 16\n              }\n            ],\n            \"height\": 16,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"CBBWnntcSAG8-dwJem0lxA\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"P7AFNDinSRSINtDM3qcN5A\",\n                \"tilesetUid\": \"KRgHcR7lQgumvyvtWzzPDA\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"darkorange\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"H3K4me1\"\n                },\n                \"width\": 468,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"ZZ2m0InZSGClbk0-Q44cBQ\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"TrPsWIpWR8uevNwwu7dTCw\",\n                \"tilesetUid\": \"XJr9O1QQQQOtf5bir2B1EA\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"darksalmon\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"H3K4me3\"\n                },\n                \"width\": 468,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"aN_anifTT5aij3XX8rsCVw\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"JKzKxBiyQBq1SaEPJ_M6Vw\",\n                \"tilesetUid\": \"c3OsZ874Tri9OpY15DT3ZA\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"cornflowerblue\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"H3K27me3\"\n                },\n                \"width\": 468,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"XHnTQY3yS3WyKwZtTWr32Q\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"cE26C5opSrW7Uoix-0MHeg\",\n                \"tilesetUid\": \"WYyBM9EyR5CZ56N14sTTdQ\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"#a31919\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"Rad21 (dRad21+IAA)\"\n                },\n                \"width\": 468,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"CD-8jw-jRgOs6tKupyvRyQ\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"O0FmvaldRf6u0j86ZEVFOw\",\n                \"tilesetUid\": \"IZRz45CxSkOYQQC4G9LdHA\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"#a31919\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"Smc1a (dRad21+IAA)\"\n                },\n                \"width\": 468,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"T7CkjdJ0T_ayonnHe44hJw\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"MnWvyy0FR2GPNuz0EH7yqA\",\n                \"tilesetUid\": \"Re5dDo5zR86fEMhwkHd17A\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"#a31919\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"CTCF (dRad21+IAA)\"\n                },\n                \"width\": 468,\n                \"height\": 25\n              }\n            ],\n            \"height\": 25,\n            \"options\": {},\n            \"width\": 468\n          }\n        ],\n        \"left\": [],\n        \"center\": [\n          {\n            \"uid\": \"Ys3jHqpwQIWXTGeNsCSVtg\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"heatmap\",\n                \"uid\": \"O1HVWyCLQ9qiUFzhuUxKZw\",\n                \"tilesetUid\": \"Yq2Fq5mbQ3qM9QWLWg_CVg\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"backgroundColor\": \"#eeeeee\",\n                  \"labelPosition\": \"topRight\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": false,\n                  \"colorRange\": [\n                    \"white\",\n                    \"rgba(245,166,35,1.0)\",\n                    \"rgba(208,2,27,1.0)\",\n                    \"black\"\n                  ],\n                  \"colorbarBackgroundColor\": \"#ffffff\",\n                  \"maxZoom\": null,\n                  \"minWidth\": 100,\n                  \"minHeight\": 100,\n                  \"colorbarPosition\": \"topRight\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"heatmapValueScaling\": \"log\",\n                  \"showMousePosition\": true,\n                  \"mousePositionColor\": \"#000000\",\n                  \"showTooltip\": false,\n                  \"extent\": \"full\",\n                  \"zeroValueColor\": null,\n                  \"name\": \"mESC dRAD21+IAA\",\n                  \"scaleStartPercent\": \"-0.00000\",\n                  \"scaleEndPercent\": \"0.08590\"\n                },\n                \"width\": 468,\n                \"height\": 584,\n                \"transforms\": [\n                  {\n                    \"name\": \"ICE\",\n                    \"value\": \"weight\"\n                  }\n                ]\n              }\n            ],\n            \"height\": 584,\n            \"options\": {},\n            \"width\": 468\n          }\n        ],\n        \"bottom\": [],\n        \"right\": [],\n        \"whole\": [],\n        \"gallery\": []\n      },\n      \"layout\": {\n        \"w\": 4,\n        \"h\": 12,\n        \"x\": 4,\n        \"y\": 0\n      },\n      \"uid\": \"MSHvIbiZRX27lXKMqY_OjA\"\n    },\n    {\n      \"initialXDomain\": [\n        1621907477.7184148,\n        1624255202.6916652\n      ],\n      \"initialYDomain\": [\n        1620837572.2374709,\n        1623217568.3443744\n      ],\n      \"tracks\": {\n        \"top\": [\n          {\n            \"type\": \"horizontal-stacked-bar\",\n            \"uid\": \"ca7f5NdbSsGUF_ne36CFEA\",\n            \"tilesetUid\": \"WmHBFc78TmOgTCIePEwawQ\",\n            \"server\": \"https://resgen.io/api/v1\",\n            \"options\": {\n              \"labelPosition\": \"topLeft\",\n              \"labelColor\": \"black\",\n              \"labelTextOpacity\": 0.4,\n              \"valueScaling\": \"exponential\",\n              \"trackBorderWidth\": 0,\n              \"trackBorderColor\": \"black\",\n              \"backgroundColor\": \"white\",\n              \"barBorder\": true,\n              \"sortLargestOnTop\": true,\n              \"selectRows\": null,\n              \"name\": \"Epilogos (mm10)\",\n              \"colorScale\": [\n                \"#0e6f37\",\n                \"#c7e4c0\",\n                \"#cdcdcd\",\n                \"#41ac5e\",\n                \"#f3eb1a\",\n                \"#f3eb1a\",\n                \"#faf8c8\",\n                \"#808080\",\n                \"#808080\",\n                \"#0454a3\",\n                \"#deecf7\",\n                \"#4290cf\",\n                \"#f48c8f\",\n                \"#fde2e5\",\n                \"#ffffff\"\n              ]\n            },\n            \"width\": 448,\n            \"height\": 20\n          },\n          {\n            \"uid\": \"KgmNtAD8RM-FZPXiTGQuNw\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"chromosome-labels\",\n                \"uid\": \"Lm1q786QQgixJMOm5sLjWA\",\n                \"tilesetUid\": \"O8tQOpn0RjiFrUBmrhph4A\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"color\": \"#808080\",\n                  \"stroke\": \"#ffffff\",\n                  \"fontSize\": 12,\n                  \"fontIsLeftAligned\": false,\n                  \"showMousePosition\": false,\n                  \"mousePositionColor\": \"#000000\"\n                },\n                \"width\": 448,\n                \"height\": 30\n              }\n            ],\n            \"height\": 30,\n            \"options\": {},\n            \"width\": 448\n          },\n          {\n            \"uid\": \"JePzgKgsSeS9tRGqCJDZ8Q\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"horizontal-gene-annotations\",\n                \"uid\": \"cPxh3DfvS8mOWHPghMz2HQ\",\n                \"tilesetUid\": \"DVuS61VhTG6DxKhlsTssTQ\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"fontSize\": 8,\n                  \"labelColor\": \"black\",\n                  \"labelBackgroundColor\": \"#ffffff\",\n                  \"labelPosition\": \"hidden\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"minHeight\": 24,\n                  \"plusStrandColor\": \"blue\",\n                  \"minusStrandColor\": \"red\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"showMousePosition\": false,\n                  \"mousePositionColor\": \"#000000\",\n                  \"geneAnnotationHeight\": 8,\n                  \"geneLabelPosition\": \"outside\",\n                  \"geneStrandSpacing\": 4,\n                  \"name\": \"Gene Annotations (mm10)\"\n                },\n                \"width\": 448,\n                \"height\": 80\n              }\n            ],\n            \"height\": 80,\n            \"options\": {},\n            \"width\": 448\n          },\n          {\n            \"uid\": \"S53La_2GRvebvoSuSJXvww\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"WNHKWanuQIG7Oth3N-QQsA\",\n                \"tilesetUid\": \"YeXV8q2TQ8-jSgsUa3d1HA\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"gold\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"H3K27ac\",\n                  \"showMousePosition\": true\n                },\n                \"width\": 448,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 448\n          },\n          {\n            \"uid\": \"Ig3676J3Tmm3LOjXCHrjGA\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"eGvJ3lU0R_CqO-LqTkltyA\",\n                \"tilesetUid\": \"KRgHcR7lQgumvyvtWzzPDA\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"darkorange\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"H3K4me1\"\n                },\n                \"width\": 448,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 448\n          },\n          {\n            \"uid\": \"BFsh3Vc-RFSlE96nxlRj3w\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"Y0Wd3xEtQJWEcXNEQwnUXg\",\n                \"tilesetUid\": \"XJr9O1QQQQOtf5bir2B1EA\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"darksalmon\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"H3K4me3\"\n                },\n                \"width\": 448,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 448\n          },\n          {\n            \"uid\": \"Hc_K1vNYS8yTyKWRhW7y_w\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"axFjLUKiRouDWqpmhUTkRw\",\n                \"tilesetUid\": \"c3OsZ874Tri9OpY15DT3ZA\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"cornflowerblue\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"H3K27me3\"\n                },\n                \"width\": 448,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 448\n          },\n          {\n            \"uid\": \"La7nXwcKTF6kjmUco2hcZg\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"UGW7WF1hSMGyXwVSeKH1yQ\",\n                \"tilesetUid\": \"AmOK9PA2TPCW2ufS00-_qQ\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelBackgroundColor\": \"white\",\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"lineStrokeColor\": \"blue\",\n                  \"lineStrokeWidth\": 1,\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"showMousePosition\": false,\n                  \"minHeight\": 20,\n                  \"mousePositionColor\": \"#000000\",\n                  \"showTooltip\": false,\n                  \"name\": \"Rad21\",\n                  \"align\": \"bottom\",\n                  \"barFillColor\": \"#a31919\",\n                  \"barOpacity\": 1\n                },\n                \"width\": 448,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 448\n          },\n          {\n            \"uid\": \"Md3xya1uTvaE0xLwH-4d6Q\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"DWQAQTOnRomhtwts4P_joA\",\n                \"tilesetUid\": \"d4XbaEjyS3uhGfZpjBuHbQ\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"#a31919\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"Smc1a\"\n                },\n                \"width\": 448,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 448\n          },\n          {\n            \"uid\": \"Yg08_90lShuHzdZjuB4-nw\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"TjGrBftdRNqz_jUyZX4fMA\",\n                \"tilesetUid\": \"C7wIL827QzCgNx0A6Y63uQ\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"#a31919\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"CTCF\"\n                },\n                \"width\": 448,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 448\n          }\n        ],\n        \"left\": [\n          {\n            \"uid\": \"VDjsCGikTN64QIF73E4LYg\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"FLhIOofFR1qhLaABLdFLOg\",\n                \"tilesetUid\": \"c3OsZ874Tri9OpY15DT3ZA\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"bottomRight\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": null,\n                  \"barFillColor\": \"cornflowerblue\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"H3K27me3\"\n                },\n                \"width\": 20,\n                \"height\": 590\n              }\n            ],\n            \"height\": 590,\n            \"width\": 20,\n            \"options\": {}\n          }\n        ],\n        \"center\": [\n          {\n            \"uid\": \"YT0VqFj3R3K4DCDqOI9zxg\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"heatmap\",\n                \"uid\": \"MJDUIH6BTJ-BGNvVu2TqVA\",\n                \"tilesetUid\": \"Cj5GErm3SRaZOiBqHDuaqg\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"backgroundColor\": \"transparent\",\n                  \"labelPosition\": \"topRight\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": false,\n                  \"colorRange\": [\n                    \"white\",\n                    \"rgba(245,166,35,1.0)\",\n                    \"rgba(208,2,27,1.0)\",\n                    \"black\"\n                  ],\n                  \"colorbarBackgroundColor\": \"#ffffff\",\n                  \"maxZoom\": null,\n                  \"minWidth\": 100,\n                  \"minHeight\": 100,\n                  \"colorbarPosition\": \"topRight\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"heatmapValueScaling\": \"log\",\n                  \"showMousePosition\": true,\n                  \"mousePositionColor\": \"#000000\",\n                  \"showTooltip\": false,\n                  \"extent\": \"full\",\n                  \"zeroValueColor\": null,\n                  \"name\": \"mESC untreated merged\",\n                  \"scaleStartPercent\": \"-0.00000\",\n                  \"scaleEndPercent\": \"0.13392\"\n                },\n                \"width\": 448,\n                \"height\": 590,\n                \"transforms\": [\n                  {\n                    \"name\": \"ICE\",\n                    \"value\": \"weight\"\n                  }\n                ]\n              }\n            ],\n            \"height\": 590,\n            \"options\": {},\n            \"width\": 448\n          }\n        ],\n        \"bottom\": [],\n        \"right\": [],\n        \"whole\": [],\n        \"gallery\": []\n      },\n      \"layout\": {\n        \"w\": 4,\n        \"h\": 12,\n        \"x\": 0,\n        \"y\": 0\n      },\n      \"uid\": \"G6n1Y3-NRG6BkREOx5T75g\"\n    },\n    {\n      \"initialXDomain\": [\n        1621867138.8010347,\n        1624295541.609045\n      ],\n      \"initialYDomain\": [\n        1620835555.2913098,\n        1623219585.2905357\n      ],\n      \"tracks\": {\n        \"top\": [\n          {\n            \"type\": \"horizontal-stacked-bar\",\n            \"uid\": \"I_bGdIGyS2Wadku4L48Z_A\",\n            \"tilesetUid\": \"WmHBFc78TmOgTCIePEwawQ\",\n            \"server\": \"https://resgen.io/api/v1\",\n            \"options\": {\n              \"labelPosition\": \"topLeft\",\n              \"labelColor\": \"black\",\n              \"labelTextOpacity\": 0.4,\n              \"valueScaling\": \"exponential\",\n              \"trackBorderWidth\": 0,\n              \"trackBorderColor\": \"black\",\n              \"backgroundColor\": \"white\",\n              \"barBorder\": true,\n              \"sortLargestOnTop\": true,\n              \"selectRows\": null,\n              \"name\": \"Epilogos (mm10)\",\n              \"colorScale\": [\n                \"#0e6f37\",\n                \"#c7e4c0\",\n                \"#cdcdcd\",\n                \"#41ac5e\",\n                \"#f3eb1a\",\n                \"#f3eb1a\",\n                \"#faf8c8\",\n                \"#808080\",\n                \"#808080\",\n                \"#0454a3\",\n                \"#deecf7\",\n                \"#4290cf\",\n                \"#f48c8f\",\n                \"#fde2e5\",\n                \"#ffffff\"\n              ]\n            },\n            \"width\": 468,\n            \"height\": 20\n          },\n          {\n            \"uid\": \"KgmNtAD8RM-FZPXiTGQuNw\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"chromosome-labels\",\n                \"uid\": \"Lm1q786QQgixJMOm5sLjWA\",\n                \"tilesetUid\": \"O8tQOpn0RjiFrUBmrhph4A\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"color\": \"#808080\",\n                  \"stroke\": \"#ffffff\",\n                  \"fontSize\": 12,\n                  \"fontIsLeftAligned\": false,\n                  \"showMousePosition\": false,\n                  \"mousePositionColor\": \"#000000\"\n                },\n                \"width\": 468,\n                \"height\": 30\n              }\n            ],\n            \"height\": 30,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"JePzgKgsSeS9tRGqCJDZ8Q\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"horizontal-gene-annotations\",\n                \"uid\": \"cPxh3DfvS8mOWHPghMz2HQ\",\n                \"tilesetUid\": \"DVuS61VhTG6DxKhlsTssTQ\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"fontSize\": 8,\n                  \"labelColor\": \"black\",\n                  \"labelBackgroundColor\": \"#ffffff\",\n                  \"labelPosition\": \"hidden\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"minHeight\": 24,\n                  \"plusStrandColor\": \"blue\",\n                  \"minusStrandColor\": \"red\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"showMousePosition\": false,\n                  \"mousePositionColor\": \"#000000\",\n                  \"geneAnnotationHeight\": 8,\n                  \"geneLabelPosition\": \"outside\",\n                  \"geneStrandSpacing\": 4,\n                  \"name\": \"Gene Annotations (mm10)\"\n                },\n                \"width\": 468,\n                \"height\": 79\n              }\n            ],\n            \"height\": 79,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"S53La_2GRvebvoSuSJXvww\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"WNHKWanuQIG7Oth3N-QQsA\",\n                \"tilesetUid\": \"YeXV8q2TQ8-jSgsUa3d1HA\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"gold\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"H3K27ac\"\n                },\n                \"width\": 468,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"WyLqV4zHTmCqSGBumZKz4A\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"X7rAgFALSNCLgwAB2ktXiA\",\n                \"tilesetUid\": \"KRgHcR7lQgumvyvtWzzPDA\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"darkorange\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"H3K4me1\"\n                },\n                \"width\": 468,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"GHhzuryhSriaiw-h5D0YQw\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"aVz6L9ReTvurjtB8O6i4qA\",\n                \"tilesetUid\": \"XJr9O1QQQQOtf5bir2B1EA\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"darksalmon\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"H3K4me3\"\n                },\n                \"width\": 468,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"Hc_K1vNYS8yTyKWRhW7y_w\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"axFjLUKiRouDWqpmhUTkRw\",\n                \"tilesetUid\": \"c3OsZ874Tri9OpY15DT3ZA\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"cornflowerblue\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"H3K27me3\"\n                },\n                \"width\": 468,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"eX7QXr7ZTzaplupeGnTl_w\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"blrwneC-Tv6OqMReqBBRsQ\",\n                \"tilesetUid\": \"Sqov3SnURXOaKqqnXjxB8Q\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelBackgroundColor\": \"white\",\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"lineStrokeColor\": \"blue\",\n                  \"lineStrokeWidth\": 1,\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"showMousePosition\": false,\n                  \"minHeight\": 20,\n                  \"mousePositionColor\": \"#000000\",\n                  \"showTooltip\": false,\n                  \"name\": \"Rad21 (dCTCF+IAA)\",\n                  \"align\": \"bottom\",\n                  \"barFillColor\": \"#a31919\",\n                  \"barOpacity\": 1\n                },\n                \"width\": 468,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"E_daKhqLR3y1eKjEbV9b-g\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"S7YKTxtfTcWtIECTL6K-rw\",\n                \"tilesetUid\": \"Y3qhU9NoS0-wFse9POW9hQ\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"#a31919\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"Smc1a (dCTCF+IAA)\"\n                },\n                \"width\": 468,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 468\n          },\n          {\n            \"uid\": \"DYaasSCeTxyNjJ1NbYANFQ\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"bar\",\n                \"uid\": \"Vdgf21fBRFWfDOkj2tXvIQ\",\n                \"tilesetUid\": \"KjN4v5lYQIOloGztRF-ciQ\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"align\": \"bottom\",\n                  \"labelColor\": \"black\",\n                  \"labelPosition\": \"topLeft\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": false,\n                  \"labelShowAssembly\": true,\n                  \"axisLabelFormatting\": \"scientific\",\n                  \"axisPositionHorizontal\": \"right\",\n                  \"barFillColor\": \"#a31919\",\n                  \"valueScaling\": \"linear\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"labelTextOpacity\": 1,\n                  \"barOpacity\": 1,\n                  \"name\": \"CTCF (dCTCF+IAA)\"\n                },\n                \"width\": 468,\n                \"height\": 20\n              }\n            ],\n            \"height\": 20,\n            \"options\": {},\n            \"width\": 468\n          }\n        ],\n        \"left\": [],\n        \"center\": [\n          {\n            \"uid\": \"HjzPk4paRqig94vWKjFyQg\",\n            \"type\": \"combined\",\n            \"contents\": [\n              {\n                \"type\": \"heatmap\",\n                \"uid\": \"DJKlZlqQTiGxrIyi9FPjBA\",\n                \"tilesetUid\": \"WPmAeHkiSM-rGgH2cp00FQ\",\n                \"server\": \"https://resgen.io/api/v1\",\n                \"options\": {\n                  \"backgroundColor\": \"#eeeeee\",\n                  \"labelPosition\": \"topRight\",\n                  \"labelLeftMargin\": 0,\n                  \"labelRightMargin\": 0,\n                  \"labelTopMargin\": 0,\n                  \"labelBottomMargin\": 0,\n                  \"labelShowResolution\": true,\n                  \"labelShowAssembly\": true,\n                  \"colorRange\": [\n                    \"white\",\n                    \"rgba(245,166,35,1.0)\",\n                    \"rgba(208,2,27,1.0)\",\n                    \"black\"\n                  ],\n                  \"colorbarBackgroundColor\": \"#ffffff\",\n                  \"maxZoom\": null,\n                  \"minWidth\": 100,\n                  \"minHeight\": 100,\n                  \"colorbarPosition\": \"topRight\",\n                  \"trackBorderWidth\": 0,\n                  \"trackBorderColor\": \"black\",\n                  \"heatmapValueScaling\": \"log\",\n                  \"showMousePosition\": true,\n                  \"mousePositionColor\": \"#000000\",\n                  \"showTooltip\": false,\n                  \"extent\": \"full\",\n                  \"zeroValueColor\": null,\n                  \"name\": \"mESC dCTCF+IAA\",\n                  \"scaleStartPercent\": \"0.00001\",\n                  \"scaleEndPercent\": \"0.08969\"\n                },\n                \"width\": 468,\n                \"height\": 591,\n                \"transforms\": [\n                  {\n                    \"name\": \"ICE\",\n                    \"value\": \"weight\"\n                  }\n                ]\n              }\n            ],\n            \"height\": 591,\n            \"options\": {},\n            \"width\": 468\n          }\n        ],\n        \"bottom\": [],\n        \"right\": [],\n        \"whole\": [],\n        \"gallery\": []\n      },\n      \"layout\": {\n        \"w\": 4,\n        \"h\": 12,\n        \"x\": 8,\n        \"y\": 0\n      },\n      \"uid\": \"fTAVxqWnRZiXjEhy4hnvmg\"\n    }\n  ],\n  \"zoomLocks\": {\n    \"locksByViewUid\": {\n      \"G6n1Y3-NRG6BkREOx5T75g\": \"R3TjGMtBQmaVhs0XPutj0g\",\n      \"MSHvIbiZRX27lXKMqY_OjA\": \"R3TjGMtBQmaVhs0XPutj0g\",\n      \"fTAVxqWnRZiXjEhy4hnvmg\": \"R3TjGMtBQmaVhs0XPutj0g\"\n    },\n    \"locksDict\": {\n      \"R3TjGMtBQmaVhs0XPutj0g\": {\n        \"fTAVxqWnRZiXjEhy4hnvmg\": [\n          1620059744.3152335,\n          1619932526.735275,\n          2082.2107322216034\n        ],\n        \"G6n1Y3-NRG6BkREOx5T75g\": [\n          1620059744.3152335,\n          1619932526.7352748,\n          2082.2107317447662\n        ],\n        \"MSHvIbiZRX27lXKMqY_OjA\": [\n          1620059744.3152335,\n          1619932526.7352748,\n          2082.2107317447662\n        ],\n        \"uid\": \"R3TjGMtBQmaVhs0XPutj0g\"\n      }\n    }\n  },\n  \"locationLocks\": {\n    \"locksByViewUid\": {\n      \"G6n1Y3-NRG6BkREOx5T75g\": \"b3ReUHGUQLK5PV-R2SW63g\",\n      \"MSHvIbiZRX27lXKMqY_OjA\": \"b3ReUHGUQLK5PV-R2SW63g\",\n      \"fTAVxqWnRZiXjEhy4hnvmg\": \"b3ReUHGUQLK5PV-R2SW63g\"\n    },\n    \"locksDict\": {\n      \"b3ReUHGUQLK5PV-R2SW63g\": {\n        \"fTAVxqWnRZiXjEhy4hnvmg\": [\n          1620059744.3152335,\n          1619932526.735275,\n          2082.2107322216034\n        ],\n        \"G6n1Y3-NRG6BkREOx5T75g\": [\n          1620059744.3152335,\n          1619932526.7352748,\n          2082.2107317447662\n        ],\n        \"MSHvIbiZRX27lXKMqY_OjA\": [\n          1620059744.3152335,\n          1619932526.7352748,\n          2082.2107317447662\n        ],\n        \"uid\": \"b3ReUHGUQLK5PV-R2SW63g\"\n      }\n    }\n  },\n  \"valueScaleLocks\": {\n    \"locksByViewUid\": {\n      \"fTAVxqWnRZiXjEhy4hnvmg.Vdgf21fBRFWfDOkj2tXvIQ\": \"GRU-x59qTECOVv0RVpJEcQ\",\n      \"MSHvIbiZRX27lXKMqY_OjA.MnWvyy0FR2GPNuz0EH7yqA\": \"GRU-x59qTECOVv0RVpJEcQ\",\n      \"G6n1Y3-NRG6BkREOx5T75g.TjGrBftdRNqz_jUyZX4fMA\": \"GRU-x59qTECOVv0RVpJEcQ\",\n      \"fTAVxqWnRZiXjEhy4hnvmg.S7YKTxtfTcWtIECTL6K-rw\": \"AEjwOXf_RJa4Ar3y7D7g_Q\",\n      \"fTAVxqWnRZiXjEhy4hnvmg.blrwneC-Tv6OqMReqBBRsQ\": \"IxrPdLIwSE-gegmrvFw1Lg\",\n      \"MSHvIbiZRX27lXKMqY_OjA.cE26C5opSrW7Uoix-0MHeg\": \"IxrPdLIwSE-gegmrvFw1Lg\",\n      \"G6n1Y3-NRG6BkREOx5T75g.UGW7WF1hSMGyXwVSeKH1yQ\": \"IxrPdLIwSE-gegmrvFw1Lg\",\n      \"MSHvIbiZRX27lXKMqY_OjA.O0FmvaldRf6u0j86ZEVFOw\": \"AEjwOXf_RJa4Ar3y7D7g_Q\",\n      \"G6n1Y3-NRG6BkREOx5T75g.DWQAQTOnRomhtwts4P_joA\": \"AEjwOXf_RJa4Ar3y7D7g_Q\"\n    },\n    \"locksDict\": {\n      \"GRU-x59qTECOVv0RVpJEcQ\": {\n        \"G6n1Y3-NRG6BkREOx5T75g.TjGrBftdRNqz_jUyZX4fMA\": {\n          \"view\": \"G6n1Y3-NRG6BkREOx5T75g\",\n          \"track\": \"TjGrBftdRNqz_jUyZX4fMA\"\n        },\n        \"fTAVxqWnRZiXjEhy4hnvmg.Vdgf21fBRFWfDOkj2tXvIQ\": {\n          \"view\": \"fTAVxqWnRZiXjEhy4hnvmg\",\n          \"track\": \"Vdgf21fBRFWfDOkj2tXvIQ\"\n        },\n        \"MSHvIbiZRX27lXKMqY_OjA.MnWvyy0FR2GPNuz0EH7yqA\": {\n          \"view\": \"MSHvIbiZRX27lXKMqY_OjA\",\n          \"track\": \"MnWvyy0FR2GPNuz0EH7yqA\"\n        },\n        \"uid\": \"GRU-x59qTECOVv0RVpJEcQ\"\n      },\n      \"AEjwOXf_RJa4Ar3y7D7g_Q\": {\n        \"fTAVxqWnRZiXjEhy4hnvmg.S7YKTxtfTcWtIECTL6K-rw\": {\n          \"view\": \"fTAVxqWnRZiXjEhy4hnvmg\",\n          \"track\": \"S7YKTxtfTcWtIECTL6K-rw\"\n        },\n        \"MSHvIbiZRX27lXKMqY_OjA.O0FmvaldRf6u0j86ZEVFOw\": {\n          \"view\": \"MSHvIbiZRX27lXKMqY_OjA\",\n          \"track\": \"O0FmvaldRf6u0j86ZEVFOw\"\n        },\n        \"G6n1Y3-NRG6BkREOx5T75g.DWQAQTOnRomhtwts4P_joA\": {\n          \"view\": \"G6n1Y3-NRG6BkREOx5T75g\",\n          \"track\": \"DWQAQTOnRomhtwts4P_joA\"\n        },\n        \"uid\": \"AEjwOXf_RJa4Ar3y7D7g_Q\"\n      },\n      \"IxrPdLIwSE-gegmrvFw1Lg\": {\n        \"G6n1Y3-NRG6BkREOx5T75g.UGW7WF1hSMGyXwVSeKH1yQ\": {\n          \"view\": \"G6n1Y3-NRG6BkREOx5T75g\",\n          \"track\": \"UGW7WF1hSMGyXwVSeKH1yQ\"\n        },\n        \"fTAVxqWnRZiXjEhy4hnvmg.blrwneC-Tv6OqMReqBBRsQ\": {\n          \"view\": \"fTAVxqWnRZiXjEhy4hnvmg\",\n          \"track\": \"blrwneC-Tv6OqMReqBBRsQ\"\n        },\n        \"MSHvIbiZRX27lXKMqY_OjA.cE26C5opSrW7Uoix-0MHeg\": {\n          \"view\": \"MSHvIbiZRX27lXKMqY_OjA\",\n          \"track\": \"cE26C5opSrW7Uoix-0MHeg\"\n        },\n        \"uid\": \"IxrPdLIwSE-gegmrvFw1Lg\"\n      }\n    }\n  }\n}",
    "created_by_name": "emilynav",
    "project": "YcBYitV8TX6ll4UE9fqrOw",
    "project_name": "Hsieh2021bioRxiv",
    "project_gruser": "mirnylab",
    "image_uuid": "Z3JezPybR42rT9eXCGd_3Q",
    "image_thumb_uuid": None,
    "name": "Hoxb zoomed out",
    "description": "",
    "created": "2025-04-13T19:10:27.936114Z"
}['viewconf'])

{
  "editable": true,
  "trackSourceServers": [
    "/api/v1",
    "http://higlass.io/api/v1"
  ],
  "exportViewUrl": "/api/v1/viewconfs",
  "views": [
    {
      "initialXDomain": [
        1621867138.8013206,
        1624295541.6087596
      ],
      "initialYDomain": [
        1620849673.912563,
        1623205466.669282
      ],
      "tracks": {
        "top": [
          {
            "type": "horizontal-stacked-bar",
            "uid": "PtVf0DYWTwCnGrGW9AkqWg",
            "tilesetUid": "WmHBFc78TmOgTCIePEwawQ",
            "server": "https://resgen.io/api/v1",
            "options": {
              "labelPosition": "topLeft",
              "labelColor": "black",
              "labelTextOpacity": 0.4,
              "valueScaling": "exponential",
              "trackBorderWidth": 0,
              "trackBorderColor": "black",
              "backgroundColor": "white",
              "barBorder": true,
              "sortLargestOnTop": true,
              "selectRows": null,
       

In [3]:
import clodius.tiles.bam as ctb
import pysam

filename = '../data/SRR1770413.sorted.short.bam'
index_filename = '../data/SRR1770413.sorted.short.bam.bai'

f = pysam.AlignmentFile(filename, index_filename=index_filename)

In [19]:
reads = list(f.fetch('Chromosome', 1, 10000))

In [21]:
%%time

for read in reads[:150]:
    print(read.cigarstring)
    print([r for r in read.get_aligned_pairs(with_seq=True) if r[2] is not None and r[2].islower()])

167H134M
[(93, 93, 'a'), (98, 98, 'a')]
30S271M
[]
183H118M
[]
37S264M
[(299, 262, 'a')]
301M
[]
134S167M
[(195, 61, 'g')]
37S264M
[(246, 209, 'c'), (262, 225, 'a'), (269, 232, 'c'), (283, 246, 'g')]
153H148M
[]
252H49M
[]
229H72M
[]
224H77M
[]
51S250M
[(217, 166, 'a'), (267, 216, 'a'), (268, 217, 'c'), (279, 228, 'a')]
117S184M
[]
56S245M
[(183, 127, 'a')]
47S254M
[]
257H44M
[(43, 43, 'a')]
53S248M
[(289, 236, 'a'), (292, 239, 't')]
47S254M
[]
149S152M
[]
209H92M
[]
143S158M
[]
41S260M
[]
131S170M
[(137, 6, 't'), (153, 22, 'g')]
154H147M
[(112, 112, 'g')]
58S243M
[(116, 58, 't')]
92S209M
[]
57S244M
[(69, 12, 't')]
210H91M
[]
157H144M
[(81, 81, 't')]
229H72M
[]
26S275M
[(40, 14, 'a'), (54, 28, 't'), (64, 38, 't'), (187, 161, 'g'), (212, 186, 't'), (279, 253, 'g')]
87S214M
[]
263H38M
[]
172H129M
[]
103S198M
[]
9S292M
[]
124S177M
[]
47S254M
[]
267H34M
[]
92S209M
[(226, 134, 'a')]
131S170M
[]
236H65M
[]
8S293M
[]
301M
[(31, 32, 't')]
301M
[]
12S289M
[]
301M
[(291, 294, 't')]
301M
[(226, 2

In [31]:
reads[1].cigartuples

[(4, 30), (0, 271)]

In [30]:
reads[1].cigarstring

'30S271M'

301

In [55]:
def get_cigar_substitutions(read):
    subs = []
    curr_pos = 0
    
    cigartuples = read.cigartuples
    readstart = read.pos
    readend = read.pos + read.query_length
    
    for ctuple in cigartuples:
        if ctuple[0] == pysam.CDIFF:
            subs.append((None, readstart + curr_pos, 'X', ctuple[1]))
            curr_pos += ctuple[1]
        elif ctuple[0] == pysam.CINS:
            subs.append((None, readstart + curr_pos, 'I', ctuple[1]))
        elif ctuple[0] == pysam.CDEL:
            subs.append((None, readstart + curr_pos, 'D', ctuple[1]))
            curr_pos += ctuple[1]
        elif ctuple[0] == pysam.CREF_SKIP:
            subs.append((None, readstart + curr_pos, 'N', ctuple[1]))
            curr_pos += ctuple[1]
        elif ctuple[0] == pysam.CEQUAL or ctuple[0] == pysam.CMATCH:
            curr_pos += ctuple[1]
    
    if len(cigartuples):
        first_ctuple = cigartuples[0]
        last_ctuple = cigartuples[-1]
        
        if first_ctuple[0] == pysam.CSOFT_CLIP:
            subs.append((None, readstart-first_ctuple[1], 'S', first_ctuple[1]))
        if first_ctuple[0] == pysam.CHARD_CLIP:
            subs.append((None, readstart-first_ctuple[1], 'H', first_ctuple[1])) 

        if last_ctuple[0] == pysam.CSOFT_CLIP:
            subs.append((None, readend, 'S', last_ctuple[1]))
        if last_ctuple[0] == pysam.CHARD_CLIP:
            subs.append((None, readend, 'H', last_ctuple[1])) 

    return subs

In [57]:
get_cigar_substitutions(reads[1])

[(None, -30, 'S', 30)]